In [1]:
import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
%matplotlib inline

In [2]:
root_path="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/database"

In [3]:
def mfcc_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features

In [4]:
#NOT USED
def fft_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    fft = np.fft.fft(signal)
    
    return fft

In [5]:
#NOT USED
signal, sample_rate = librosa.load('database/ac/ac_AHCG_FSRPNCW.wav')
ipd.Audio('database/ac/ac_AHCG_FSRPNCW.wav')
fft = np.fft.fft(signal)
fft

array([ 3.96959355+0.j        , -1.57368256-0.43014124j,
       -0.5562213 +0.51962332j, ...,  0.00995858+0.12657352j,
       -0.5562213 -0.51962332j, -1.57368256+0.43014124j])

In [6]:
#NOT USED
signal, sample_rate = librosa.load('database\sol\sol_AKZI_FAWCVUT.wav' )
ipd.Audio('database\sol\sol_AKZI_FAWCVUT.wav' )
mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
mfccs_features.shape

(40, 44)

In [7]:
extracted_feataures =[]
for root, directories, files in os.walk(root_path): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names))
            data = mfcc_extractor(os.path.join(root,file_names))
            extracted_feataures.append([data,os.path.basename(os.path.normpath(root))])

In [ ]:
extracted_feataures_dataframe= pd.DataFrame(extracted_feataures, columns=["mfcc", "word"])
extracted_feataures_dataframe.head()

,mfcc,word
0,"[-466.59473, 54.1608, -32.242916, 11.20391, -1...",ac
1,"[-445.2056, 60.57163, -23.187899, 16.521847, -...",ac
2,"[-419.66544, 68.09416, -13.22027, 10.260144, -...",ac
3,"[-483.0338, 62.193104, -10.424236, 17.89255, -...",ac
4,"[-393.0876, 61.467396, -20.944004, 18.326431, ...",ac


In [ ]:
X=np.array(extracted_feataures_dataframe['mfcc'].tolist())
Y=np.array(extracted_feataures_dataframe['word'].tolist())

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
Y_numerical=to_categorical(labelencoder.fit_transform(Y))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y_numerical, test_size=0.2, random_state=0)

In [ ]:
#NOT USED
X_train.shape

(21187, 40)

In [ ]:
#NOT USED
len(Y_test)

5297

ANN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
model= Sequential()

model.add(Flatten(input_shape=(40, )))
model.add(Dense(300, Activation('relu')))
model.add(Dropout(0.2))
model.add(Dense(300, Activation('relu')))
model.add(Dense(Y_numerical.shape[1], Activation('softmax')))

"""
model.add(Dense(200, input_shape=(40,)))
model.add(Activation('relu'))

model.add(Dense(300))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(Y_numerical.shape[1]))
model.add(Activation('sigmoid'))"""

"\nmodel.add(Dense(200, input_shape=(40,)))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(300))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(50))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(Y_numerical.shape[1]))\nmodel.add(Activation('sigmoid'))"

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 40)                0         
                                                                 
 dense_4 (Dense)             (None, 300)               12300     
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense_5 (Dense)             (None, 300)               90300     
                                                                 
 dense_6 (Dense)             (None, 14)                4214      
                                                                 
Total params: 106,814
Trainable params: 106,814
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [ ]:
model.fit(X_train, Y_train, epochs=70, validation_data=(X_test, Y_test), verbose=2)

Epoch 1/200
663/663 - 2s - loss: 0.4652 - accuracy: 0.8393 - val_loss: 0.9198 - val_accuracy: 0.7408 - 2s/epoch - 3ms/step
Epoch 2/200
663/663 - 2s - loss: 0.4627 - accuracy: 0.8398 - val_loss: 0.8955 - val_accuracy: 0.7414 - 2s/epoch - 3ms/step
Epoch 3/200
663/663 - 2s - loss: 0.4669 - accuracy: 0.8385 - val_loss: 0.9216 - val_accuracy: 0.7427 - 2s/epoch - 3ms/step
Epoch 4/200
663/663 - 2s - loss: 0.4591 - accuracy: 0.8377 - val_loss: 0.9035 - val_accuracy: 0.7448 - 2s/epoch - 3ms/step
Epoch 5/200
663/663 - 2s - loss: 0.4560 - accuracy: 0.8394 - val_loss: 0.9345 - val_accuracy: 0.7334 - 2s/epoch - 3ms/step
Epoch 6/200
663/663 - 2s - loss: 0.4699 - accuracy: 0.8358 - val_loss: 0.9385 - val_accuracy: 0.7353 - 2s/epoch - 3ms/step
Epoch 7/200
663/663 - 2s - loss: 0.4612 - accuracy: 0.8397 - val_loss: 0.9125 - val_accuracy: 0.7442 - 2s/epoch - 3ms/step
Epoch 8/200
663/663 - 2s - loss: 0.4535 - accuracy: 0.8414 - val_loss: 0.9538 - val_accuracy: 0.7310 - 2s/epoch - 3ms/step
Epoch 9/200
663/

In [ ]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 0s 2ms/step - loss: 1.1317 - accuracy: 0.7440
Test Score:  [1.1316728591918945, 0.7440060377120972]
663/663 [==============================] - 1s 2ms/step - loss: 0.0961 - accuracy: 0.9693
Train Score:  [0.0960678681731224, 0.9692736268043518]


PREDICTION WITH DATA

In [ ]:
predictiondata_feataures =[]
counter_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/Records"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicted_label=np.argmax(model.predict(rl), axis=-1)
            prediction_class = labelencoder.inverse_transform(predicted_label) 
            predictiondata_feataures.append([os.path.basename(os.path.normpath(root)), prediction_class, predicted_label])
            if os.path.basename(os.path.normpath(root))==prediction_class:
                counter_true+=1

In [ ]:
predictiondata_feataures_dataframe= pd.DataFrame(predictiondata_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentage= (counter_true/len(predictiondata_feataures))*100
print("The percantage of the true predicted data: ", percentage,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondata_feataures_dataframe


The percantage of the true predicted data:  50.0 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[ac],[0]
2,ac,[ac],[0]
3,asagi,[asagi],[1]
4,asagi,[asagi],[1]
5,asagi,[asagi],[1]
6,baslat,[baslat],[2]
7,baslat,[baslat],[2]
8,baslat,[baslat],[2]
9,devam,[sag],[11]


In [ ]:
"""filename="predictionData/dursultan.wav"
r=mfcc_extractor(filename)
print(r)
rl=r.reshape(1,-1)
print(rl)
print(rl.shape)
predicted_label=np.argmax(model.predict(rl), axis=-1)

print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class"""

'filename="predictionData/dursultan.wav"\nr=mfcc_extractor(filename)\nprint(r)\nrl=r.reshape(1,-1)\nprint(rl)\nprint(rl.shape)\npredicted_label=np.argmax(model.predict(rl), axis=-1)\n\nprint(predicted_label)\nprediction_class = labelencoder.inverse_transform(predicted_label) \nprediction_class'